# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [67]:
import numpy as np

import pandas as pd
import os
import sys
import time
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [68]:
# Checking Python environment in use

In [69]:
sys.executable

'C:\\Users\\Prithvi\\anaconda3\\envs\\udemyllm\\python.exe'

In [70]:
os.getcwd()

'C:\\Users\\Prithvi\\Downloads\\Practice\\Udemy - LLM Engineering\\llm_engineering\\week2'

In [71]:
os.environ.get('CONDA_DEFAULT_ENV')

'udemyllm'

In [72]:
load_dotenv(override = True)

True

In [73]:
# Loading OPENAI_API_KEY

In [74]:
openai_api_key = os.getenv('OPENAI_API_KEY')

In [75]:
openai_api_key[:4]

'sk-p'

In [76]:
openai = OpenAI()

In [127]:
import json
import gradio as gr
import openai

# ------------------ System Message ------------------
system_message = """
You are a helpful AI assistant which always gives courteous replies.
If you don't know the answer, politely say so.
When showing booking details, ALWAYS return a clean and beautiful Markdown table.
"""

MODEL = "gpt-4.1"  # Use GPT-4.1 or GPT-4.1-mini if available

# ------------------ Ticket Price Tool ------------------
ticket_prices = {
    "london": "$799",
    "paris": "$899",
    "tokyo": "$1400",
    "berlin": "$499"
}

def get_ticket_price(destination_city):
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to"
            }
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

# ------------------ Flight Booking Tool ------------------
def book_flight(name, origin, destination, travel_date):
    return {
        "confirmation_message": "Your flight is confirmed! 🎉",
        "details": {
            "Passenger Name": name,
            "Origin": origin,
            "Destination": destination,
            "Travel Date": travel_date,
            "Booking Status": "✅ Confirmed"
        }
    }

booking_function = {
    "name": "book_flight",
    "description": "Book a flight and confirm the booking. Returns passenger details and booking status.",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {"type": "string", "description": "Passenger's full name"},
            "origin": {"type": "string", "description": "Departure city"},
            "destination": {"type": "string", "description": "Arrival city"},
            "travel_date": {"type": "string", "description": "Date of travel in YYYY-MM-DD format"}
        },
        "required": ["name", "origin", "destination", "travel_date"],
        "additionalProperties": False
    }
}

# ------------------ Tools ------------------
tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": booking_function}
]

# ------------------ Tool Handler ------------------
def handle_tool_call(tool_call):
    """Executes the tool and returns the tool message."""
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    if function_name == "get_ticket_price":
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        response_content = {"destination_city": city, "price": price}

    elif function_name == "book_flight":
        name = arguments.get('name')
        origin = arguments.get('origin')
        destination = arguments.get('destination')
        travel_date = arguments.get('travel_date')
        booking_info = book_flight(name, origin, destination, travel_date)
        response_content = booking_info

    return {
        "role": "tool",
        "tool_call_id": tool_call.id,  # REQUIRED for OpenAI
        "content": json.dumps(response_content)
    }

# ------------------ Chat Function ------------------
def chat_openai(user_message, history):
    # Build OpenAI-style messages from history
    messages = [{"role": "system", "content": system_message}]
    for h in history:
        messages.append(h)
    messages.append({"role": "user", "content": user_message})

    # First response
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools
    )

    message = response.choices[0].message

    # If a tool is called, handle it
    if response.choices[0].finish_reason == "tool_calls":
        tool_call = message.tool_calls[0]
        tool_response = handle_tool_call(tool_call)

        # Append assistant + tool response to messages
        messages.append(message)  # assistant asking for tool
        messages.append(tool_response)  # our tool's reply

        # Final response after tool execution
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools
        )
        message = response.choices[0].message

    # Update history in OpenAI's format
    history.append({"role": "user", "content": user_message})
    history.append({"role": "assistant", "content": message.content})
    return "", history, history

# ------------------ Gradio UI ------------------
with gr.Blocks() as demo:
    gr.Markdown("## ✈️ Flight Booking Assistant")
    chatbot = gr.Chatbot(label="AI Assistant", type="messages", height=500)
    msg = gr.Textbox(label="Type your message...")
    clear = gr.Button("Clear Chat")
    state = gr.State([])

    msg.submit(chat_openai, [msg, state], [msg, chatbot, state])
    clear.click(lambda: [], None, [chatbot, state], queue=False)

demo.launch()


* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.
